In [94]:
import pandas as pd
import numpy as np
import sys
import os
from os import system
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
import statistics
from statistics import mode

In [95]:
data_dir = "data"
for root, dirs, files in os.walk(data_dir, topdown=False):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))

data/title.basics.tsv/data.tsv
data/data-sarc-sample/sarc/sarcastic_879.txt
data/data-sarc-sample/sarc/sarcastic_223.txt
data/data-sarc-sample/sarc/sarcastic_318.txt
data/data-sarc-sample/sarc/sarcastic_118.txt
data/data-sarc-sample/sarc/sarcastic_932.txt
data/data-sarc-sample/sarc/sarcastic_129.txt
data/data-sarc-sample/sarc/sarcastic_494.txt
data/data-sarc-sample/sarc/sarcastic_662.txt
data/data-sarc-sample/sarc/sarcastic_592.txt
data/data-sarc-sample/sarc/sarcastic_135.txt
data/data-sarc-sample/sarc/sarcastic_470.txt
data/data-sarc-sample/sarc/sarcastic_87.txt
data/data-sarc-sample/sarc/sarcastic_209.txt
data/data-sarc-sample/sarc/sarcastic_869.txt
data/data-sarc-sample/sarc/sarcastic_743.txt
data/data-sarc-sample/sarc/sarcastic_952.txt
data/data-sarc-sample/sarc/sarcastic_933.txt
data/data-sarc-sample/sarc/sarcastic_904.txt
data/data-sarc-sample/sarc/sarcastic_230.txt
data/data-sarc-sample/sarc/sarcastic_784.txt
data/data-sarc-sample/sarc/sarcastic_915.txt
data/data-sarc-sample/sar

In [96]:
# Combine to TSV
if 'data' not in os.getcwd(): 
    os.chdir("data")
system("pwd")
if not os.path.exists('combined.csv'):
    system("cat *.csv > combined.csv")
os.chdir("../")

# Convert to TSV here

/home/pavle/Documents/Git/medvidov/dsci550/data


In [97]:
pix_df = pd.read_csv("data/new_file_with_features.csv")
pix_df.sort_values(by=['Account Created Date'])
pix_df['Gender'] = pix_df['Gender'].astype("str") # Change type of column
# pix_df['Age'] = pix_df['Age'].astype("int32") # Change type of column
pix_df

,Unnamed: 0.1,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,...,contains_GLAAD_hate_speech,contains_ADL_hate_speech,Posted during Covid Peak Year,Belong to High Covid Mortality Risk Age Group,"Belong to High Respiratory Disease (Pneumonia, Influenza, or COVID-19) Mortality Risk Age Group",Relevant_film_event,24-hour Views,24-hour Comments,Tags,Relevant Sports Event(s)
0,0,0,156581,STY1666112066,86625,USR1662300553,nan,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",...,False,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O..."
1,1,1,156580,STY1666111954,119236,USR1665943314,nan,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",...,False,False,False,False,False,"('Haifa Film Festival', 'Newport Beach Film Fe...",5843845,11933,(),"['Womens World Cup', 'World Track Championship..."
2,2,2,156579,STY1666111923,86566,USR1662294899,nan,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",...,False,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O..."
3,3,3,156578,STY1666111798,86655,USR1662303432,nan,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",...,False,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O..."
4,4,4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",...,False,False,True,False,False,"('Haifa International Film Festival', 'Zurich ...",0,0,(),['World Cup (women)']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,94995,120365,STY1659205745,33023,USR1650524692,male,12,Open House Session on Sustainability in Archit...,Earth Day Network-India Director Climate Chang...,...,False,False,False,False,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']"
94996,94996,94996,120364,STY1659205388,33023,USR1650524692,male,12,EarthDayMorningAssemblyContest2022- MCD School...,Earth Day Network-India congratulates MCD Scho...,...,False,False,False,False,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']"
94997,94997,94997,120363,STY1659205364,9256,USR1632837537,female,20,Lonavala,Often called the 'Jewel of Sahyadri' and the '...,...,False,False,True,False,False,"('Zurich Film Festival',)",0,0,(),['World Cup (women)']
94998,94998,94998,120362,STY1659205224,20802,USR1642092657,male,33,"Calcio, Ã¨ fatta per De Ketelaere al Milan: 32...",Lâobiettivo di mercato tanto ricercato dal M...,...,False,False,False,False,False,(),1583029,1399,"('politics',)",['Africa Cup of Nations']


In [98]:
# ADD THIS CELL
# Sarcasm classification using corpus V2
# Following this tutorial: https://thecleverprogrammer.com/2021/08/24/sarcasm-detection-with-machine-learning/

# Access sarcasm data directory
sarc_data_dir = "data/sarcasm_v2/"

# Placeholder df to get all sorts of sarcasm
sarc_df = pd.DataFrame()

# Get all types of sarcasm into the df
for root, dirs, files in os.walk(sarc_data_dir, topdown=False):
    for name in files:
        df = pd.read_csv(os.path.join(root, name))
        sarc_df = pd.concat([sarc_df, df])

# Group by the class and change sarc and notsarc to 1s and 0s
sarc_df.groupby('class')
sarc_df = sarc_df.replace('notsarc',0)
sarc_df = sarc_df.replace('sarc',1)
sarc_df

,class,id,text
0,0,1,"Archie, the ONLY issue that gays don't have a ..."
1,0,2,"No, not really. All that is different is the n..."
2,0,3,It's ashame that everyone keeps looking for th...
3,0,4,"Almost? Usually, that is true, and it involves..."
4,0,5,And so have animals. Plants have been wiped ou...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [99]:
# ADD THIS CELL
# Define X and Y
x = np.array(sarc_df["text"])
y = np.array(sarc_df["class"])

# Count vectorizer for sarcasm detection
cv = CountVectorizer()

# Fit and transform the data and make stratified train and test sets
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify = y)

# Classify and score on the data. Using ComplementNB because it sometimes outperforms MultinomialNB
# bern = BernoulliNB()
# bern.fit(X_train, y_train)
# mult = MultinomialNB()
# mult.fit(X_train, y_train)
comp = ComplementNB()
comp.fit(X_train, y_train)
# print(bern.score(X_test, y_test))
# print(mult.score(X_test, y_test)) # Notably this scores the same as ComplementNB
print(comp.score(X_test, y_test))

0.7374866879659212


In [100]:
# ADD THIS CELL
# Create new column to append
sarcasm = []

# Classify sarcasm using ComplementNB
for i in pix_df.iterrows():
    text = [np.str_(i[1]['Narrative'])]
    data = cv.transform(text)
    output = comp.predict(data)
    sarcasm.append(output[0]) # Add sarcasm classification to column

# Add sarcasm column to df
pix_df['Sarcasm'] = sarcasm

In [101]:
pix_df

,Unnamed: 0.1,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,...,contains_ADL_hate_speech,Posted during Covid Peak Year,Belong to High Covid Mortality Risk Age Group,"Belong to High Respiratory Disease (Pneumonia, Influenza, or COVID-19) Mortality Risk Age Group",Relevant_film_event,24-hour Views,24-hour Comments,Tags,Relevant Sports Event(s),Sarcasm
0,0,0,156581,STY1666112066,86625,USR1662300553,nan,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",...,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0
1,1,1,156580,STY1666111954,119236,USR1665943314,nan,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",...,False,False,False,False,"('Haifa Film Festival', 'Newport Beach Film Fe...",5843845,11933,(),"['Womens World Cup', 'World Track Championship...",0
2,2,2,156579,STY1666111923,86566,USR1662294899,nan,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",...,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0
3,3,3,156578,STY1666111798,86655,USR1662303432,nan,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",...,False,False,False,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0
4,4,4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",...,False,True,False,False,"('Haifa International Film Festival', 'Zurich ...",0,0,(),['World Cup (women)'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,94995,120365,STY1659205745,33023,USR1650524692,male,12,Open House Session on Sustainability in Archit...,Earth Day Network-India Director Climate Chang...,...,False,False,False,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']",0
94996,94996,94996,120364,STY1659205388,33023,USR1650524692,male,12,EarthDayMorningAssemblyContest2022- MCD School...,Earth Day Network-India congratulates MCD Scho...,...,False,False,False,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']",0
94997,94997,94997,120363,STY1659205364,9256,USR1632837537,female,20,Lonavala,Often called the 'Jewel of Sahyadri' and the '...,...,False,True,False,False,"('Zurich Film Festival',)",0,0,(),['World Cup (women)'],0
94998,94998,94998,120362,STY1659205224,20802,USR1642092657,male,33,"Calcio, Ã¨ fatta per De Ketelaere al Milan: 32...",Lâobiettivo di mercato tanto ricercato dal M...,...,False,False,False,False,(),1583029,1399,"('politics',)",['Africa Cup of Nations'],0


In [102]:
# ADD THIS CELL
# Filtering based on IMDB data

# Create title data df
title_data = "data/title.basics.tsv/data.tsv"
title_df = pd.read_csv(title_data, sep="\t")
title_df = title_df.drop(labels = ['originalTitle','runtimeMinutes', 'isAdult','endYear'], axis = 1) # Drop useless columns

# Turn release year to an int
title_df['startYear']= title_df['startYear'].replace("\\N","-1") # Replace non-number values
title_df['startYear'] = title_df['startYear'].astype("int64") # Change type of column

# Filter title_df by timeframe
title_df = title_df[title_df['startYear']>=2020]
title_df = title_df[title_df['startYear']<=2022]
title_df = title_df[title_df['titleType'] == 'movie']
title_df = title_df[title_df['genres'] != "\\N"] # Ensure genres exist

title_df

/tmp/ipykernel_6388/349661114.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_df = pd.read_csv(title_data, sep="\t")


,tconst,titleType,primaryTitle,startYear,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,2021,Documentary
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,2022,Documentary
102656,tt0104988,movie,Neues in Wittstock,2021,Documentary
117002,tt0119830,movie,One Dog Day,2022,"Action,Comedy"
...,...,...,...,...,...
9639793,tt9914192,movie,No Gogó do Paulinho,2020,Comedy
9640153,tt9914972,movie,Blind Ambition,2021,Documentary
9640683,tt9916190,movie,Safeguard,2020,"Action,Adventure,Thriller"
9640722,tt9916270,movie,Il talento del calabrone,2020,Thriller


In [103]:
# ADD THIS CELL
# Create pix_df for comparison to find movies
comp_pix_df = pix_df[pix_df['Interest'].str.contains('movie|film|hollywood',regex=True)]
comp_pix_df

,Unnamed: 0.1,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,...,contains_ADL_hate_speech,Posted during Covid Peak Year,Belong to High Covid Mortality Risk Age Group,"Belong to High Respiratory Disease (Pneumonia, Influenza, or COVID-19) Mortality Risk Age Group",Relevant_film_event,24-hour Views,24-hour Comments,Tags,Relevant Sports Event(s),Sarcasm
8,8,8,156573,STY1666111207,11447,USR1637905680,female,22,âEverybody from Starland Was Thereâ: The H...,"<p dir=""ltr"">Remembering the Oct. 18, 1922 ope...",...,False,True,False,False,"('Tallinn Black Nights Film Festival', 'Intern...",0,0,(),"['World Cup', 'South-East Asian Games', 'World...",0
17,17,17,156564,STY1666110639,11447,USR1637905680,female,22,Emma Watson Debuts New Pixie Cut,"<p dir=""ltr"">Emma Watson debuted a new pixie c...",...,False,True,False,False,"('Tallinn Black Nights Film Festival', 'Intern...",0,0,(),"['World Cup', 'South-East Asian Games', 'World...",0
20,20,20,156561,STY1666110226,11447,USR1637905680,female,22,Â Man of Steel sequel starring Henry Cavill as...,"<p dir=""ltr"">Henry Cavill is set to reprise hi...",...,False,True,False,False,"('Tallinn Black Nights Film Festival', 'Intern...",0,0,(),"['World Cup', 'South-East Asian Games', 'World...",0
22,22,22,156559,STY1666109729,59836,USR1658242492,nan,23,Amber Heard Moved To Spain As An Escape From H...,"<p dir=""ltr"">Amber Heard Moved To Spain As An ...",...,True,False,False,False,"('New York Asian Film Festival', 'Fantasia Fil...",37877963,12806,(),"['Tour de France', 'IAAF World Championships']",0
23,23,23,156558,STY1666109571,11447,USR1637905680,female,22,Dua Lipa looks like Megan Fox in new pics,"<p dir=""ltr"">Time to update the list of celebr...",...,False,True,False,False,"('Tallinn Black Nights Film Festival', 'Intern...",0,0,(),"['World Cup', 'South-East Asian Games', 'World...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94923,94923,94923,120437,STY1659247596,11859,USR1638425715,male,21,à¦ à¦­à¦¿à¦¨à§à¦¤à¦¾ à¦°à¦£à¦¬à§à¦° à¦à¦¾à¦...,https://bengali.nenow.in/entertainment/fire-br...,...,False,True,False,False,(),0,0,(),"['Special Olympics World Winter Games', 'Six N...",0
94928,94928,94928,120432,STY1659246709,11859,USR1638425715,male,21,à¦à¦¤à§à¦¤à¦°à¦ªà§à¦°à§à¦¬à§à¦° à¦®à¦¾à¦¨...,https://bengali.nenow.in/top-news/delhi-police...,...,False,True,False,False,(),0,0,(),"['Special Olympics World Winter Games', 'Six N...",0
94933,94933,94933,120427,STY1659245094,26991,USR1644249897,nan,44,Late night snack date night!,Watching a movie with the wife is always bette...,...,False,False,False,False,"('Bentonville Film Festival', 'Karlovy Vary In...",0,0,(),"['World Aquatics Championships', '2021 Summer ...",0
94947,94947,94947,120413,STY1659236275,11045,USR1637258628,others,22,"Malaika Arora cheers for ""handsome' beau, Arju...","A few hours back, Arjun Kapoor walked the ramp...",...,False,True,False,False,"('European Film Forum SCANORAMA', 'London Kore...",0,0,(),['World Cup'],1


In [105]:
# ADD THIS CELL
# Filter pix_df so my PC can run this
female_pix_df = comp_pix_df[comp_pix_df['Gender']=='female']
male_pix_df = comp_pix_df[comp_pix_df['Gender']=='male']
other_pix_df = comp_pix_df[(comp_pix_df['Gender']!='female') | (comp_pix_df['Gender']!='male')]
pix_df_PG13 = comp_pix_df[comp_pix_df['Age'] < 17]
pix_df_R = comp_pix_df[comp_pix_df['Age'] >= 17]

In [106]:
# ADD THIS CELL
# Get rating data
rating_data = "data/title.ratings.tsv/data.tsv"
rating_df = pd.read_csv(rating_data, sep="\t")

# Merge rating data with title data to be able to filter by ratings
merged_df = pd.merge(title_df, rating_df, on='tconst', how='outer')

# Clean merged data and filter for movies rated 8 or higher
merged_df = merged_df.replace(pd.NA,"-1")
merged_df = merged_df.replace(np.nan, "-1")
merged_df = merged_df.astype({'startYear': 'int64', 'averageRating': 'float64'})
merged_df = merged_df[merged_df["startYear"] != -1]
merged_df = merged_df[merged_df["averageRating"] >= 8]
merged_df = merged_df[merged_df["titleType"]=="movie"]

# Filter the merged_df for movies that have titles that are at least 4 characters
for index, row in merged_df.iterrows():
    if len(row['primaryTitle']) < 4:
        merged_df.drop(index, inplace=True)

merged_df

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
3,tt0104988,movie,Neues in Wittstock,2021,Documentary,8.4,8.0
9,tt0211266,movie,Black Star: Autobiography of a Close Friend,2022,Documentary,8.4,14.0
17,tt0306971,movie,Hun Hunshi Hunshilal,2021,"Comedy,Drama",8.5,52.0
23,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,2020,Documentary,8.3,36.0
53,tt10011240,movie,Cinematti - Una storia folle,2020,Documentary,9.0,12.0
...,...,...,...,...,...,...,...
50425,tt9851854,movie,Major,2022,"Action,Biography,Drama",8.1,29669.0
50448,tt9866708,movie,Wild Karnataka,2020,"Documentary,Family",8.7,668.0
50451,tt9867478,movie,Approaching Shadows,2021,Horror,8.2,6.0
50478,tt9891640,movie,Sergio Mendes in the Key of Joy,2020,"Biography,Documentary,Music",8.1,41.0


In [107]:
# ADD THIS CELL

title_list = []
genre_list = []

# Get most popular title and genre for females
for i in female_pix_df.iterrows():
    for j in merged_df.iterrows():
        if j[1]['primaryTitle'] in i[1]['Narrative']:
            title_list.append(j[1]['primaryTitle'])
            genre_list.append(j[1]['genres'])
            
female_title = mode(title_list)
female_genre = mode(genre_list)

In [108]:
# ADD THIS CELL

title_list = []
genre_list = []

# Get most popular title and genre for males
for i in male_pix_df.iterrows():
    for j in merged_df.iterrows():
        if j[1]['primaryTitle'] in i[1]['Narrative']:
            title_list.append(j[1]['primaryTitle'])
            genre_list.append(j[1]['genres'])
            
male_title = mode(title_list)
male_genre = mode(genre_list)

In [109]:
# ADD THIS CELL

title_list = []
genre_list = []

# Get most popular title and genre for unspecified and other genders
for i in other_pix_df.iterrows():
    for j in merged_df.iterrows():
        if j[1]['primaryTitle'] in i[1]['Narrative']:
            title_list.append(j[1]['primaryTitle'])
            genre_list.append(j[1]['genres'])
            
other_title = mode(title_list)
other_genre = mode(genre_list)

In [110]:
# ADD THIS CELL

title_list = []
genre_list = []

# Get most popular title and genre for people who cannot watch rated R movies
for i in pix_df_PG13.iterrows():
    for j in merged_df.iterrows():
        if j[1]['primaryTitle'] in i[1]['Narrative']:
            title_list.append(j[1]['primaryTitle'])
            genre_list.append(j[1]['genres'])
            
PG13_title = mode(title_list)
PG13_genre = mode(genre_list)

In [111]:
# ADD THIS CELL

title_list = []
genre_list = []

# Get most popular title and genre for people who can watch rated R movies
for i in pix_df_R.iterrows():
    for j in merged_df.iterrows():
        if j[1]['primaryTitle'] in i[1]['Narrative']:
            title_list.append(j[1]['primaryTitle'])
            genre_list.append(j[1]['genres'])
            
R_title = mode(title_list)
R_genre = mode(genre_list)

In [113]:
# ADD THIS CELL

title_by_gender = []
genre_by_gender = []
genre_by_age = []

for i in pix_df.iterrows():
    
    if i[1]['Gender'] == 'female':
        title_by_gender.append(female_title)
        genre_by_gender.append(female_genre)
    elif i[1]['Gender'] == 'male':
        title_by_gender.append(male_title)
        genre_by_gender.append(male_genre)
    else:
        title_by_gender.append(other_title)
        genre_by_gender.append(other_genre)
        
    if i[1]['Age'] < 17:
        genre_by_age.append(PG13_genre)
    else:
        genre_by_age.append(R_genre)

pix_df['Predicted Movie by Gender'] = title_by_gender
pix_df['Predicted Genre by Gender'] = genre_by_gender
pix_df['Predicted Genre by Age'] = genre_by_age

In [114]:
pix_df

,Unnamed: 0.1,Unnamed: 0,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,...,"Belong to High Respiratory Disease (Pneumonia, Influenza, or COVID-19) Mortality Risk Age Group",Relevant_film_event,24-hour Views,24-hour Comments,Tags,Relevant Sports Event(s),Sarcasm,Predicted Movie by Gender,Predicted Genre by Gender,Predicted Genre by Age
0,0,0,156581,STY1666112066,86625,USR1662300553,nan,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",...,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0,Vikram,Drama,Drama
1,1,1,156580,STY1666111954,119236,USR1665943314,nan,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",...,False,"('Haifa Film Festival', 'Newport Beach Film Fe...",5843845,11933,(),"['Womens World Cup', 'World Track Championship...",0,Vikram,Drama,Drama
2,2,2,156579,STY1666111923,86566,USR1662294899,nan,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",...,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0,Vikram,Drama,Drama
3,3,3,156578,STY1666111798,86655,USR1662303432,nan,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",...,False,"('Venice International Film Festival', 'Deauvi...",275837,197,(),"['World Volleyball Championships (men)', 'US O...",0,Vikram,Drama,Drama
4,4,4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",...,False,"('Haifa International Film Festival', 'Zurich ...",0,0,(),['World Cup (women)'],1,Director,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,94995,120365,STY1659205745,33023,USR1650524692,male,12,Open House Session on Sustainability in Archit...,Earth Day Network-India Director Climate Chang...,...,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']",0,Director,Drama,Drama
94996,94996,94996,120364,STY1659205388,33023,USR1650524692,male,12,EarthDayMorningAssemblyContest2022- MCD School...,Earth Day Network-India congratulates MCD Scho...,...,False,"('Seattle International Film Festival', 'Fanta...",253058,282,"('politics', 'entertainment')","['Invictus Games', 'World Snooker Championship']",0,Director,Drama,Drama
94997,94997,94997,120363,STY1659205364,9256,USR1632837537,female,20,Lonavala,Often called the 'Jewel of Sahyadri' and the '...,...,False,"('Zurich Film Festival',)",0,0,(),['World Cup (women)'],0,Vikram,Drama,Drama
94998,94998,94998,120362,STY1659205224,20802,USR1642092657,male,33,"Calcio, Ã¨ fatta per De Ketelaere al Milan: 32...",Lâobiettivo di mercato tanto ricercato dal M...,...,False,(),1583029,1399,"('politics',)",['Africa Cup of Nations'],0,Director,Drama,Drama


In [116]:
pix_df.to_csv('data/new_file_with_features.csv')
with open('data/new_file_with_features.csv','r', encoding='utf-8') as csvin2, open('data/complete.tsv', 'w', newline='', encoding='utf-8') as tsvout2:
    csvin2 = csv.reader(csvin2)
    tsvout2 = csv.writer(tsvout2, delimiter='\t')

    for row in csvin2:
        tsvout2.writerow(row)